# Preamble


In [ ]:
%load_ext autoreload
%autoreload 2
%aimport
# %reload_ext autoreload

import sys 
from os.path import abspath
path = abspath('..')
if path not in sys.path: 
    sys.path.insert(0, path)
print(path)

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
from astropy.table import Table, Column

In [ ]:
from relaxed import utils
from relaxed.frames import params, catalogs
from relaxed.subhalos import subhalo

from pminh import minh

In [ ]:
cat_filepath = Path('/home/imendoza/alcca/nbody-relaxed/data/phils_catalogues/'
                    'halo_catalogues/Bolshoi/BolshoiP/minh/hlist_1.00000.minh')
name = 'BolshoiP'

# try minh

In [ ]:
mcat = minh.open(cat_filepath)

In [ ]:
print(mcat.names)

In [ ]:
t = Table(mcat.read(('id', 'mvir')), names=['id', 'mvir'])

In [ ]:
mcat.read([])

# try hcat 

In [ ]:
hcat = catalogs.HaloCatalog(cat_filepath, name, verbose=True)

In [ ]:
hcat.load_base_cat()

In [ ]:
hcat._cat.sort('id')

In [ ]:
hcat._cat

In [ ]:
assert np.all(hcat._cat['upid']==-1)
assert np.all(hcat._cat['mvir']>=np.log10(hcat.particle_mass * 1e3))
print(len(hcat))

# try subhalos 

## test

In [ ]:
from relaxed.subhalos import subhalo

In [ ]:
host_ids = np.array([100, 200, 50, -17], dtype=int)
sub_pids = np.array([100, -17, 100, 200, -17, 100, 75, 300, -20], dtype=int)
sub_mvir = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=float)

In [ ]:
order = np.argsort(host_ids)
host_ids = host_ids[order]

In [ ]:
host_ids

In [ ]:
sub_index = np.searchsorted(host_ids, sub_pids)

In [ ]:
sub_index

In [ ]:
sub_ok = sub_index < len(host_ids)
sub_ok

In [ ]:
sub_ok[sub_ok] &= host_ids[sub_index[sub_ok]] == sub_pids[sub_ok]

out = np.ones(len(sub_index), dtype=int) * -1
out[sub_ok] = self.host_index[sub_index[sub_ok]]

In [ ]:
subhalo.m_sub(host_ids, sub_pids, sub_mvir)

## my cat 

In [ ]:
hcat = catalogs.HaloCatalog(cat_filepath, name, verbose=True, extract_sub=True)

In [ ]:
hcat.load_base_cat()

In [ ]:
plt.hist(hcat._cat['f_sub'][:-2],bins=50);

In [ ]:
cat = hcat._cat
host_ids = cat['id']
ids, upid, mvir = mcat.block(0, ['id','upid', 'mvir'])
ids = ids[upid!=-1]
sub_pids = upid[upid != -1]
sub_mvir = mvir[upid != -1]

In [ ]:
set(host_ids)

In [ ]:
len(sub_pids)

In [ ]:
table = subhalo.LookupTable(host_ids)
table.lookup(sub_pids)

In [ ]:
max(host_ids)

In [ ]:
max(sub_pids)